In [3]:
import pandas as pd

In [4]:
taxi = pd.read_csv('https://stepik.org/media/attachments/lesson/360340/2_taxi_nyc.csv')

In [5]:
taxi.head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


In [50]:
taxi.shape

(29101, 14)

In [51]:
taxi.dtypes

pickup_dt        object
pickup_month     object
borough          object
pickups           int64
hday             object
spd             float64
vsb             float64
temp            float64
dewp            float64
slp             float64
pcp_01          float64
pcp_06          float64
pcp_24          float64
sd              float64
dtype: object

# В названиях столбцов pcp 01, pcp 06, pcp 24 встречается пробел.

Это не очень удобно, если в дальнейшем вы планируете обращаться к колонкам через точку, без использования кавычек и скобочек. 

Замените пробел в названиях на знак нижнего подчёркивания. 

In [52]:
taxi = taxi.rename(columns={'pcp 01':'pcp_01', 'pcp 06':'pcp_06', 'pcp 24':'pcp_24'})

# Вас попросили узнать, сколько записей (строк) в датафрейме относятся к району Манхэттен (Manhattan). 

Получить ответ на данный вопрос можно было бы, например, с помощью следующей команды:

taxi.query("borough == 'Manhattan'").shape[0]

А если мы хотим посмотреть, сколько раз встречается каждый из районов? Неужели придётся использовать подобную конструкцию для каждого уровня переменной? Здесь на помощь приходит метод value_counts(), подробнее о котором можно почитать в конспекте! 

А теперь вопрос: сколько раз в данных встречается район Бруклин (Brooklyn)?

In [97]:
taxi.query("borough == 'Brooklyn'").shape[0]

4343

In [7]:
min_pickups = taxi.groupby('borough').aggregate({'pickups':'sum'}).idxmin()
min_pickups

pickups    EWR
dtype: object

# Продолжим изучение данных и посмотрим на число поездок в выходные дни. 

Сгруппируйте данные по двум признакам: району города и является ли день выходным (колонки borough и hday). Сравните среднее число поездок, и выберите районы, из которых по праздникам в среднем поступает больше заказов, чем в обычные дни. 

In [96]:
holiday = taxi.query('hday == "Y"').groupby(['borough'], as_index = False).agg({'pickups':'mean'})
weekday = taxi.query('hday == "N"').groupby(['borough'], as_index = False).agg({'pickups':'mean'})

for i in range(len(holiday)):
    if holiday.pickups[i] > weekday.pickups[i]:
        print(holiday.borough[i]) 

EWR
Queens


# Для каждого района посчитайте число поездок по месяцам. Отсортируйте полученные значения по убыванию и сохраните результирующий датафрейм в pickups_by_mon_bor.

Обратите внимание, что итоговый датасет должен состоять из трёх колонок: pickup_month, borough, pickups.

In [103]:
pickups_by_mon_bor = taxi.groupby(['pickup_month', 'borough'], as_index = False) \
                         .agg({'pickups':'sum'}) \
                         .sort_values('pickups', ascending=False)

pickups_by_mon_bor

,pickup_month,borough,pickups
21,Jun,Manhattan,1995388
33,May,Manhattan,1888800
9,Feb,Manhattan,1718571
27,Mar,Manhattan,1661261
3,Apr,Manhattan,1648278
15,Jan,Manhattan,1455543
19,Jun,Brooklyn,482466
31,May,Brooklyn,476087
1,Apr,Brooklyn,378095
25,Mar,Brooklyn,346726


# 💀 Сложное задание! 💀

Теперь задача посложнее! Попрактикуемся в написании собственных функций и их применении к датафрейму. Поскольку данные о поездках в Нью-Йорке, температура представлена в градусах Фаренгейта. 

Напишите функцию temp_to_celcius, которая получает на вход колонку с температурой в °F и возвращает значения, переведённые в градусы Цельсия.

Формула:

 $ Celsius=\frac{(Fahrenheit−32)⋅5.0}{9.0} $

Пример работы:

> taxi['temp'][:5]

0    30.0
1    30.0
3    30.0
4    30.0
5    30.0

> taxi['temp_C'] = temp_to_celcius(taxi['temp'])
> taxi['temp_C'][:5]

>>0   -1.111111
1   -1.111111
3   -1.111111
4   -1.111111
5   -1.111111

In [109]:
def temp_to_celcius(temp):
    fahr=[]
    for i in temp:
        fahr.append(((i-32)*5.0)/9.0)
    return (fahr)

In [110]:
taxi['temp'][:5]

0    30.0
1    30.0
2    30.0
3    30.0
4    30.0
Name: temp, dtype: float64

In [111]:
taxi['temp_C'] = temp_to_celcius(taxi['temp'])
taxi['temp_C'][:5]

0   -1.111111
1   -1.111111
2   -1.111111
3   -1.111111
4   -1.111111
Name: temp_C, dtype: float64